In [1]:
import pandas as pd
import sqlite3

df = pd.read_csv("ecommerce_cleaned.csv")
conn = sqlite3.connect(":memory:")

df.to_sql("orders", conn, index=False, if_exists="replace")

5000

In [2]:
query = """
SELECT customer_id,
       order_id,
       total_amount,
       ROW_NUMBER() OVER (
           PARTITION BY customer_id
           ORDER BY total_amount DESC
       ) AS row_num
FROM orders;
"""
pd.read_sql(query, conn)

,customer_id,order_id,total_amount,row_num
0,CUST_00001,ORD_000001,219.32,1
1,CUST_00002,ORD_000002,332.34,1
2,CUST_00003,ORD_000003,66.44,1
3,CUST_00004,ORD_000004,112.65,1
4,CUST_00005,ORD_000005,349.45,1
...,...,...,...,...
4995,CUST_04996,ORD_004996,129.37,1
4996,CUST_04997,ORD_004997,2660.67,1
4997,CUST_04998,ORD_004998,51.04,1
4998,CUST_04999,ORD_004999,6036.70,1


SELECT customer_id,
       order_id,
       total_amount,
       ROW_NUMBER() OVER (
           PARTITION BY customer_id
           ORDER BY total_amount DESC
       ) AS row_num
FROM orders;

In [3]:
query = """
SELECT customer_id,
       SUM(total_amount) AS total_spent,
       RANK() OVER (
           ORDER BY SUM(total_amount) DESC
       ) AS rank
FROM orders
GROUP BY customer_id;
"""
pd.read_sql(query, conn)

,customer_id,total_spent,rank
0,CUST_04705,22023.90,1
1,CUST_00061,21478.35,2
2,CUST_01984,21409.05,3
3,CUST_00525,20211.81,4
4,CUST_03721,19547.20,5
...,...,...,...
4995,CUST_03040,10.49,4996
4996,CUST_01596,8.92,4997
4997,CUST_02939,8.33,4998
4998,CUST_00120,7.89,4999


In [4]:
query = """
SELECT product_category,
       SUM(total_amount) AS revenue,
       DENSE_RANK() OVER (
           ORDER BY SUM(total_amount) DESC
       ) AS category_rank
FROM orders
GROUP BY product_category;
"""
pd.read_sql(query, conn)

,product_category,revenue,category_rank
0,Electronics,2328806.81,1
1,Home & Garden,908348.86,2
2,Sports,754563.56,3
3,Fashion,375214.93,4
4,Toys,223142.48,5
5,Beauty,156584.74,6
6,Food,96138.67,7
7,Books,72744.52,8


In [5]:
query = """
SELECT product_category,
       SUM(total_amount) AS revenue,
       SUM(SUM(total_amount)) OVER (
           ORDER BY SUM(total_amount) DESC
       ) AS cumulative_revenue
FROM orders
GROUP BY product_category;
"""
pd.read_sql(query, conn)

,product_category,revenue,cumulative_revenue
0,Electronics,2328806.81,2328806.81
1,Home & Garden,908348.86,3237155.67
2,Sports,754563.56,3991719.23
3,Fashion,375214.93,4366934.16
4,Toys,223142.48,4590076.64
5,Beauty,156584.74,4746661.38
6,Food,96138.67,4842800.05
7,Books,72744.52,4915544.57


## Key Takeaways

- Window functions perform calculations across rows without collapsing data.
- ROW_NUMBER, RANK, and DENSE_RANK are common interview topics.
- Cumulative metrics help identify revenue concentration and Pareto effects.